## Define the Siamese Model, Train and Evaluate Model 

#### Load the Images datasets

In [1]:
# Load the libraries
import os
from siamese import Siamese
from L1Dist import L1Dist
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall
    

# Create the constants for the data folders
ANC_PATH = os.path.join('data', 'anchor')
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')

# Constants
BATCH_SIZE     = 512
PRE_FETCH_SIZE = 64
LEARNING_RATE  = 1e-4
EPOCHS = 50

#### Set GPU Growth 

In order to avoid OutOfMemory error we have to use GPU power.

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

#### Load the data for the training

In [3]:
# Load 400 of the paths for each category (helps build a clean data pipeline)
anchor_itt   = tf.data.Dataset.list_files(ANC_PATH + '/*.jpg').take(5000) # type: ignore
positive_itt = tf.data.Dataset.list_files(POS_PATH + '/*.jpg').take(5000) # type: ignore
negative_itt = tf.data.Dataset.list_files(NEG_PATH + '/*.jpg').take(5000) # type: ignore

Metal device set to: Apple M2 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



#### Data Preprocessing  

In [4]:
# Normalize the image values from [0,255] to [0,1] for a better Gradient Descend process

def normalize(image_path):
    # Load image
    byte_img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(byte_img)

    # Normalize in [0,1] and resize to 100x100x3 for the model
    img = tf.image.resize(img, (100,100))
    img = img / 255.0

    return img

#### Create labeled dataset

In [5]:
# Create a dataset that includes both positive and negative examples
positive = tf.data.Dataset.zip((anchor_itt, positive_itt, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor_itt))))) # type: ignore
negative = tf.data.Dataset.zip((anchor_itt, negative_itt, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor_itt))))) # type: ignore

dataset = tf.data.Dataset.concatenate(positive, negative)

#### Create the Train-Test datasets

In [6]:
# Create a normalize function for the dataset object that has type (path, path, label)
def data_normalize(input_img, val_image, label):
    return(normalize(input_img), normalize(val_image), label)

In [7]:
# Normalize all the data of the dataset and build the data pipeline (need to shuffle)
dataset = dataset.map(data_normalize)
dataset = dataset.cache()
dataset = dataset.shuffle(buffer_size=1024)

In [8]:
# Train-Test split
train, test = tf.keras.utils.split_dataset(dataset, left_size=0.8)

# Create the batch size to train and test the model
train = train.batch(BATCH_SIZE)
train = train.prefetch(PRE_FETCH_SIZE)

test = test.batch(BATCH_SIZE)
test = test.prefetch(PRE_FETCH_SIZE)

2023-06-24 00:53:33.941876: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


### Define the Siamese Model

In [9]:
# Create the Siamese model
siamese_model = Siamese.siamese_model()
siamese_model.summary()

Model: "SiameseModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_image (InputLayer)  [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_image[0][0]',            
                                                                  'validation_image[0][

### Training

In [10]:
# Basic Info for training

# Define the loss and optimizer model
binary_cross_loss = tf.losses.BinaryCrossentropy()
opt = tf.optimizers.Adam(LEARNING_RATE)

# Create checkpoint directory
os.makedirs('training_checkpoints')
checkpoints_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoints_dir, "checkpoint")
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)


In [11]:
@tf.function
def train_step(batch):  
    # Record all of our operations 
    with tf.GradientTape() as tape:   

        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]
        
        # Forward pass
        y_pred = siamese_model(X, training=True)
        # Calculate loss
        loss = binary_cross_loss(y, y_pred)
        
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    
    # Return loss
    return loss

In [12]:
# Define the train function
def train_model(train_data, epochs):

    # Create a file with the training results
    result_file = open('results.txt', "a")

    # Loop through epochs
    for epoch in range(1, epochs+1):
        loss_per_batch = []
        print('Results for Epoch: {}'.format(epoch))
        progress_bar = tf.keras.utils.Progbar(len(train_data))

        # Creating a metric object 
        r = Recall()
        p = Precision()

        # Loop through batches
        for idx, batch in enumerate(train_data):
            loss_b = train_step(batch)
            loss_per_batch.append(loss_b)
            prediction = siamese_model.predict(batch[:2])

            # Update the recall and precision  
            r.update_state(batch[2], prediction)
            p.update_state(batch[2], prediction)
            progress_bar.update(idx+1)
        
        result_file.writelines('Loss: {0}, Precision: {1}, Recall: {2}\n'.format(loss_per_batch.pop().numpy(), r.result().numpy(), p.result().numpy()))
        print('The loss for the epoch: {0} is {1}'.format(epoch, sum(loss_per_batch)/len(loss_per_batch)))
        print('The recall for the batch is: {0}, while the precision is: {1}'.format(r.result().numpy(), p.result().numpy()))
        # Save the checkpoints
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)


In [13]:
train_model(train_data=train, epochs=EPOCHS)

Results for Epoch: 1
16/16 [==============================] - 12s 670ms/step
The loss for the epoch: 1 is 3.112259864807129
The recall for the batch is: 1.0, while the precision is: 0.6222500205039978
Results for Epoch: 2
16/16 [==============================] - 10s 645ms/step
The loss for the epoch: 2 is 0.6906810402870178
The recall for the batch is: 0.4447569251060486, while the precision is: 0.5
Results for Epoch: 3
16/16 [==============================] - 10s 645ms/step
The loss for the epoch: 3 is 0.8040938973426819
The recall for the batch is: 0.5100442171096802, while the precision is: 0.456572562456131
Results for Epoch: 4
16/16 [==============================] - 10s 647ms/step
The loss for the epoch: 4 is 0.70880126953125
The recall for the batch is: 1.0, while the precision is: 0.6222500205039978
Results for Epoch: 5
16/16 [==============================] - 11s 652ms/step
The loss for the epoch: 5 is 0.6917206048965454
The recall for the batch is: 1.0, while the precision is

### Save Model

In [14]:
siamese_model.save('siameseModel.h5')

### Reload Model

In [15]:
model = tf.keras.models.load_model('siameseModel.h5', custom_objects={'siamese': Siamese, 'L1Dist': L1Dist})
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])